## 사용하는 패키지들

In [1]:
import json
import pandas as pd
import numpy as np

from gensim.models.word2vec import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import cosine_similarity


## 데이터 로드

In [2]:
with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [3]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [4]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [5]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [6]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [7]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [8]:
train_data_sample = train_data[:500]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [9]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [10]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'여자아이돌', '걸그룹댄스', '스트레스해소', 'kpop', '댄스'}


In [11]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(16674, 2)

In [12]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [13]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [14]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   gnr                 16674 non-null  object
 3   issue_date          16674 non-null  int64 
 4   album_id            16674 non-null  int64 
 5   artist_id_basket    16674 non-null  object
 6   song_name           16674 non-null  object
 7   artist_name_basket  16674 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.1+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [15]:
train_data_sample2 = train_data[:500]

In [16]:
w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 5, workers = 4, sg = 1)

w2v.wv.vectors.shape

(66, 100)

In [17]:
print(w2v.wv.most_similar('스트레스'))

[('분위기', 0.22347375750541687), ('힙합', 0.18224726617336273), ('국힙', 0.168910413980484), ('이별', 0.16237954795360565), ('팝', 0.15214744210243225), ('휴식', 0.1397266834974289), ('여름', 0.13733252882957458), ('추억', 0.11950822919607162), ('설렘', 0.11778008937835693), ('회상', 0.1129852756857872)]


## CountVectorizer

In [37]:
song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(16674, 194)

## 유사도 측정

## 코사인 유사도

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [25]:
gnr_cos_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_cos_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.70710678,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 0.70710678, 1.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.5       ,
        1.        ]])

## 자카드 유사도

In [50]:
gnr_jac_sim = np.zeros((gnr_mat.shape[0], gnr_mat.shape[0]))

for i in range(gnr_mat.shape[0]):
    i_set = set(gnr_mat[i].indices)
    
    for j in range(i, gnr_mat.shape[0]):
        j_set = set(gnr_mat[j].indices)
        
        union = i_set.union(j_set)
        intersection = i_set.intersection(j_set)
        
        if len(union) != 0:
            gnr_jac_sim[i][j] = len(intersection)/len(union)
            gnr_jac_sim[j][i] = len(intersection)/len(union)

print(gnr_jac_sim)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.2        ... 0.         0.         0.        ]
 [0.         0.2        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.5        0.5       ]
 [0.         0.         0.         ... 0.5        1.         0.33333333]
 [0.         0.         0.         ... 0.5        0.33333333 1.        ]]


## 피어슨 유사도

In [64]:
gnr_pea_sim = np.zeros((gnr_mat.shape[0], gnr_mat.shape[0]))

for i in range(gnr_mat.shape[0]):
    a = gnr_mat[i].toarray()
    
    for j in range(i, gnr_mat.shape[0]):
        b = gnr_mat[j].toarray()
        b = b.T
        
        gnr_pea_sim[i][j] = np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))
        gnr_pea_sim[j][i] = gnr_pea_sim[i][j]

print(gnr_pea_sim)

C:\Users\user\AppData\Local\Temp\ipykernel_8300\2196435977.py:10: RuntimeWarning: invalid value encountered in true_divide
  gnr_pea_sim[i][j] = np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))


KeyboardInterrupt: 

## 모듈 1

- 태그 데이터 활용
- 태그 데이터가 주어지면 개수 부족시 w2v으로 개수를 늘리고 해당 태그가 있는 플레이리스트 반환

In [65]:
def find_tag_song(tags, songs, tag_df):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    return tag_songs

## 모듈 2

- 장르 데이터 활용

- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [66]:
def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
    
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        
        temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    
    final_index = temp.index.values[ : top_n]
    
    return df.iloc[final_index]

In [67]:
similar_songs = find_sim_song(song_tag_appended, gnr_sim, [525514, 129701, 229622], 10)
similar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]

,song_id,similarity,issue_date,gnr
11005,191430,0.761486,20140407,"[GN1402, GN1401, GN0901, GN0902]"
15445,463782,0.761486,20170227,"[GN1402, GN1401, GN0901, GN0902]"
9,205238,0.606558,20110124,"[GN0904, GN1402, GN1401, GN0901, GN1001]"
10498,701978,0.575727,20101011,"[GN1402, GN1401, GN1001]"
16038,513731,0.569036,20111018,"[GN1402, GN1401]"
16037,489449,0.569036,20111111,"[GN1402, GN1401]"
9997,392798,0.569036,20111018,"[GN1402, GN1401]"
9450,598147,0.569036,20131105,"[GN1402, GN1401]"
10908,661924,0.569036,20161021,"[GN1402, GN1401]"
695,310974,0.569036,20061107,"[GN1402, GN1401]"


## 모듈 3

- 태그와 장르 그룹에서 음악 추출
- 두 그룹 모두에 존재하는 음악을 우선으로 추출
- n개 미만인 경우 각 그룹에서 추출

In [78]:
def find_recommend_n(songs, tag_songs, simi_songs, n=10):
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 n개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < n and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 n개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < n:
        
        # 태그와 유사도 두 그룹에서 동일한 개수(홀수일 경우 태그 > 유사도) 추출
        # sc = 유사도 그룹에서 추출할 노래의 개수
        if len(recommended) % 2 == 0:
            sc = (n-len(recommended)) / 2
        else:
            sc = (n-len(recommended)) // 2
        
        # 태그는 있고 히스토리가 없는 경우 태그 그룹에서 n개를 추출하기 위한 코드
        if len(songs) == 0:
            sc = 0
        
        # 이미 추출한 노래를 제외하고 태그 그룹에서 정해진 개수만큼 추출한다
        # 태그가 없을 경우 동작하지 않음
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (n - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 그룹에서 추출한다
        index = 0
        while len(recommended) < n:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
    
    return recommended

## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [79]:
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    
    # 모듈 1 사용
    tag_songs = find_tag_song(tags, songs, tag_df)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        # 모듈 2 사용
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 모듈 3 사용
    recommended = find_recommend_n(songs, tag_songs, simi_songs)
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]

### 태그와 노래 목록 존재

In [80]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
697,146989,YOUTH,[Troye Sivan],20160123,"[나만들을꺼, 그냥, 느낌있는, 비오는날, 나만들을거얌, 질리지않는, 분위기, 가을..."
698,430106,Everglow,[Coldplay],20151204,"[감성, 휴식, Maroon, 발라드, 위대함, HipHop, 날려버려, 나만알고싶..."
706,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407,"[취향저격, 느낌있는, 질리지않는, 잔잔한, 힐링, 휴식, 신나는, 나만알고싶은, ..."
2112,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130,"[우울, 감성, 발라드, 이별, 사랑, HipHop, 새벽, 기분전환, 꿀, 귀르가..."
2114,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228,"[우울, 감성, 밤, 발라드, 감미로운, 이별, 사랑, 인생곡들만, 새벽, 기분전환..."
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 잔잔한, 힐링]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[휴가, 휴식, 책읽을때, 추억, 분위기, 잔잔한, 혼자, 주말, 감성적인]"
9,205238,"Dingue, Dingue, Dingue",[Christophe Mae],20110124,[락]
10498,701978,Meant To Be,[Rachel Belman],20101011,"[카페, 봄, 힐링]"
16038,513731,The Sun The Trees (Acoustic Ver.),[Russian Red],20111018,[팝]


### 노래 목록만 존재

In [81]:
my_tags2 = []
my_songs2 = [525514, 129701, 229622]
rec2 = song_recommend(my_tags2, my_songs2, train_data_sample2, song_tag_appended, gnr_sim)
rec2[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 잔잔한, 힐링]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[휴가, 휴식, 책읽을때, 추억, 분위기, 잔잔한, 혼자, 주말, 감성적인]"
9,205238,"Dingue, Dingue, Dingue",[Christophe Mae],20110124,[락]
10498,701978,Meant To Be,[Rachel Belman],20101011,"[카페, 봄, 힐링]"
16038,513731,The Sun The Trees (Acoustic Ver.),[Russian Red],20111018,[팝]
16037,489449,Lego House (Acoustic),[Ed Sheeran],20111111,[팝]
9997,392798,Fuerteventura,[Russian Red],20111018,"[indie, 인디, 나만의Best3, 피쉬슈즈]"
9450,598147,Take Your Time,[Louis Yoelin],20131105,"[휴식, 기분전환, 힐링]"
10908,661924,Familiar,[Agnes Obel],20161021,"[출근, 기분좋은, 팝송, 상쾌, 아침]"
695,310974,9 Crimes,[Damien Rice],20061107,"[감성, 휴식, 재즈, 까페, 뉴에이지, 피아노, 나만알고싶은, Pop, 질리지않는..."


### 태그만 존재

In [82]:
my_tags3 = ['락']
my_songs3 = []
rec3 = song_recommend(my_tags3, my_songs3, train_data_sample2, song_tag_appended, gnr_sim)
rec3[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
697,146989,YOUTH,[Troye Sivan],20160123,"[나만들을꺼, 그냥, 느낌있는, 비오는날, 나만들을거얌, 질리지않는, 분위기, 가을..."
698,430106,Everglow,[Coldplay],20151204,"[감성, 휴식, Maroon, 발라드, 위대함, HipHop, 날려버려, 나만알고싶..."
706,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407,"[취향저격, 느낌있는, 질리지않는, 잔잔한, 힐링, 휴식, 신나는, 나만알고싶은, ..."
2112,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130,"[우울, 감성, 발라드, 이별, 사랑, HipHop, 새벽, 기분전환, 꿀, 귀르가..."
2114,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228,"[우울, 감성, 밤, 발라드, 감미로운, 이별, 사랑, 인생곡들만, 새벽, 기분전환..."
3503,246531,"모든 날, 모든 순간 (Every day, Every Moment)",[폴킴],20180320,"[비_오는날, 새벽, 비오는날, 장마, 버스, 추천곡, 잔잔한, 힐링, 카페, 휴식..."
3506,450838,Slow,[SOLE (쏠)],20180713,"[카페, 감성, 그냥좋은노래, 새벽, 그리움, 겨울밤, 늦잠, 버스, 인디, 외로움..."
1872,460823,Paris In The Rain,[Lauv],20181008,"[나만들을꺼, 그냥, 카페음악, 비오는날, 나만들을거얌, 분위기, 힘든날, 밤에듣기..."
5349,543371,퇴근버스,[이준호],20140205,"[느낌있는, 새벽, 야경, 장마, 혼자있을때, 추천곡, 잔잔한, 힐링, 휴식, 인생..."
9839,116114,나의 사춘기에게,[볼빨간사춘기],20170928,"[우울, 감성, 새벽, 슬픔, 겨울밤, 좋은노래, 힘내, 추천곡, 그냥좋은노래, 힐..."


### 둘 다 없음

In [83]:
my_tags4 = []
my_songs4 = []
rec4 = song_recommend(my_tags4, my_songs4, train_data_sample2, song_tag_appended, gnr_sim)
rec4[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
89,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115,"[트로트, 겨울노래, 인디음악, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴..."
119,457519,꿀차,[우효],20180102,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
120,453762,너 정말 예쁘다,[최낙타],20180410,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
121,349398,LOVE YA!,[혁오 (HYUKOH)],20180531,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
122,631142,편지,[장희원],20180603,"[사랑, 겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, ..."
123,406082,하늘엔 별이 떠있고 너만큼은 빛나질 않아,[이민혁],20180903,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
124,548389,사랑에 빠졌네,[정준일],20181101,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
125,205179,꿀맛,[정미애],20190815,"[운동, 댄스, 연말, 요가, 분위기, 필라테스, 아무생각없이, 매장, 카페, 휴식..."
126,567076,숨겨진 세상 (Into the Unknown End Credit Version) (...,[태연 (TAEYEON)],20191115,"[겨울노래, 눈오는날, 연말, 따듯한, 겨울왕국, 크리스마스캐럴, 분위기, 캐럴, ..."
180,418694,Attention,[Charlie Puth],20180511,"[나만들을꺼, 운동, 일렉, 나만들을거얌, 질리지않는, 잔잔한, 힐링, 휴식, 나만..."
